<a href="https://colab.research.google.com/github/ck1729/ERA1/blob/main/S6/Session6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### CODE BLOCK 0: Import the required libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import time

#### CODE BLOCK 1: Build Your Network Architecture

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1), # IN=28 OUT=28 RF=3 
            nn.ReLU(),
            nn.BatchNorm2d(8),
            nn.Conv2d(8, 16, 3, padding=1), # IN=28 OUT=28 RF=5 
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1), # IN=28 OUT=28 RF=7
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2),             # IN=28 OUT=14 RF=14
            nn.Dropout(0.2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 16, 3, padding=1), # IN=14 OUT=14 RF=16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3, padding=1), # IN=14 OUT=14 RF=18
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 16, 3), # IN=14 OUT=12 RF=20
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2, 2), # IN=12 OUT=6 RF=40
            nn.Dropout(0.2)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1), # IN=6 OUT=6 RF=42
        )

        self.gap = nn.Sequential(
           nn.AvgPool2d(6)
        )
        
        self.fc = nn.Sequential(
            nn.Linear(32, 10)
        )
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x


CODE BLOCK 2: Model Summary

In [14]:
# !pip install torchsummary

In [15]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
            Conv2d-4           [-1, 16, 28, 28]           1,168
              ReLU-5           [-1, 16, 28, 28]               0
       BatchNorm2d-6           [-1, 16, 28, 28]              32
            Conv2d-7           [-1, 16, 28, 28]           2,320
              ReLU-8           [-1, 16, 28, 28]               0
       BatchNorm2d-9           [-1, 16, 28, 28]              32
        MaxPool2d-10           [-1, 16, 14, 14]               0
          Dropout-11           [-1, 16, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]           2,320
             ReLU-13           [-1, 16, 14, 14]               0
      BatchNorm2d-14           [-1, 16,

#### CODE BLOCK 3: Define your Train and Test Loaders

In [16]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        # transforms.RandomRotation(15),
                        transforms.RandomAffine(degrees=10, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ColorJitter(brightness=0.2, contrast=0.2),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

CODE BLOCK 4: Model Train and Test Functions

In [17]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'Epoch: {epoch} | Train Loss={loss.item():0.4f}')
        train_loss += loss.item()
    return train_loss
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = 100*(correct/len(test_loader.dataset))
    print(f'Test loss: {test_loss:.4f} | Test Accuracy: {100*(correct/len(test_loader.dataset)):0.2f}')
    return test_loss, test_acc

CODE BLOCK 5: Model Scheduler and Trainer

In [18]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

train_losses, test_losses = [], []
test_accuracy = []

for epoch in range(20):
    start = time.time()
    
    running_loss = 0
    accuracy=0

    train_loss = train(model, device, train_loader, optimizer, epoch)
    train_losses.append(train_loss/len(train_loader))

    test_loss, test_acc = test(model, device, test_loader)
    test_losses.append(test_loss)
    test_accuracy.append(test_accuracy)

    end = time.time()
    print(f'Time: {(end-start):0.2f} sec\n')

Epoch: 0 | Train Loss=0.1633: 100%|██████████| 469/469 [00:42<00:00, 11.04it/s]


Test loss: 0.0866 | Test Accuracy: 97.57
Time: 46.11 sec



Epoch: 1 | Train Loss=0.0482: 100%|██████████| 469/469 [00:42<00:00, 11.07it/s]


Test loss: 0.0508 | Test Accuracy: 98.36
Time: 44.93 sec



Epoch: 2 | Train Loss=0.0916: 100%|██████████| 469/469 [00:42<00:00, 10.95it/s]


...

Test loss: 0.0204 | Test Accuracy: 99.39
Time: 44.57 sec



Epoch: 18 | Train Loss=0.0490: 100%|██████████| 469/469 [00:42<00:00, 11.02it/s]


Test loss: 0.0186 | Test Accuracy: 99.39
Time: 45.10 sec



Epoch: 19 | Train Loss=0.0716: 100%|██████████| 469/469 [00:43<00:00, 10.78it/s]


Test loss: 0.0184 | Test Accuracy: 99.42
Time: 46.47 sec



In [ ]:
# plot training history
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
plt.subplot(2,1,1)
ax = plt.gca()
ax.set_xlim([0, epoch + 2])
plt.ylabel('Loss')
plt.plot(range(1, epoch + 2), train_losses[:epoch+1], 'r', label='Training Loss')
plt.plot(range(1, epoch + 2), test_losses[:epoch+1], 'b', label='Validation Loss')
ax.grid(linestyle='-.')
plt.legend()
plt.subplot(2,1,2)
ax = plt.gca()
ax.set_xlim([0, epoch+2])
plt.ylabel('Accuracy')
# plt.plot(range(1, epoch + 2), train_acc[:epoch+1], 'r', label='Training Accuracy')
# plt.plot(range(1, epoch + 2), test_accuracy[:epoch+1], 'b', label='Validation Accuracy')
ax.grid(linestyle='-.')
plt.legend()
plt.show()